# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.train.automl import AutoMLConfig

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'mlexpauto'

experiment=Experiment(ws, experiment_name)

## Creating Compute

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute

amlcompute_cluster_name = "capprocompute"

# Verify that cluster does not exist already
try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

Found existing cluster, use it.


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [5]:
factory = TabularDatasetFactory()
url = 'https://raw.githubusercontent.com/KrishnaB22/Azure-ML-Capstone/main/dataset/heart.csv'
trainds = factory.from_delimited_files(url)

In [6]:
# TODO: Put your automl settings here
automl_settings = {
    "enable_early_stopping" : True,
    "primary_metric": 'accuracy',
    "featurization": 'auto',
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(experiment_timeout_minutes=20,
                             task = 'classification',
                             compute_target=aml_compute,
                             experiment_exit_score = 0.9984,
                             blocked_models = ['KNN','LinearSVM'],
                             enable_onnx_compatible_models=True,
                             training_data = trainds,
                             label_column_name = 'target',
                             n_cross_validations=5,
                             **automl_settings
                            )

In [7]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
from azureml.widgets import RunDetails

RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [10]:
best_run_aml, fitted_model_aml = remote_run.get_output()
model_name = best_run_aml.properties['model_name']

In [11]:
#TODO: Save the best model
best_run_aml.download_file('outputs/model.pkl', 'outputs/heart-diesease-auto-model.pkl')
best_run_aml.download_file('outputs/scoring_file_v_1_0_0.py', 'outputs/score_aml.py')
best_run_aml.download_file('automl_driver.py', 'outputs/automl_driver.py')

## Auto ML Evaluation

In [12]:
import pickle
file = open("outputs/heart-diesease-auto-model.pkl",'rb')
aml_model = pickle.load(file)
file.close()

In [15]:
from train import clean_data
from sklearn.metrics import accuracy_score
import pandas as pd


factory = TabularDatasetFactory()
test_data_path = "https://raw.githubusercontent.com/KrishnaB22/Azure-ML-Capstone/main/dataset/heart.csv"
test_ds = pd.read_csv(test_data_path)
y_test = test_ds[['target']]
print(accuracy_score(aml_model.predict(test_ds.drop(columns=['target'])), y_test))

0.933993399339934


In [17]:
from sklearn.metrics import classification_report
print(classification_report(y_test, aml_model.predict(test_ds.drop(columns=['target']))))

              precision    recall  f1-score   support

           0       0.95      0.90      0.93       138
           1       0.92      0.96      0.94       165

    accuracy                           0.93       303
   macro avg       0.94      0.93      0.93       303
weighted avg       0.93      0.93      0.93       303



### The AutoML model has an acccuracy of 93%.

### As the AutoML model has higher accuracy we deploy the AutoML model 

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [36]:
from azureml.core.model import Model
model = Model.register(workspace = ws,
                        model_path ="outputs/heart-diesease-auto-model.pkl",
                        model_name = "automl_cap")

Registering model automl_cap


In [41]:
print(model)

Model(workspace=Workspace.create(name='quick-starts-ws-132450', subscription_id='6971f5ac-8af1-446e-8034-05acea24681f', resource_group='aml-quickstarts-132450'), name=automl_cap, id=automl_cap:1, version=1, tags={}, properties={})


### Inference configuration

In [45]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig


env = Environment.get(ws, "AzureML-Minimal")

for pip_package in ["scikit-learn"]:
    env.python.conda_dependencies.add_pip_package(pip_package)

inference_config = InferenceConfig(entry_script='./score.py',
                                    environment=env)

In [ ]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(ws, "mldeploy", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.......................................................

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service